In [1]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split


In [3]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['model',
 'face1.jpg',
 'face2.jpg',
 'FaceImage',
 '31_data_preprocessing.ipynb',
 'data_face_features.pickle']

In [4]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [5]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.10126288  0.04939684 -0.0135333  ...  0.00050823  0.08076832
    0.02925757]]

 [[ 0.11484744  0.06094224  0.00180208 ... -0.01531954  0.12997243
    0.02931331]]

 [[ 0.0883371   0.07687377 -0.01829172 ...  0.03242559  0.09040125
    0.00474092]]

 ...

 [[ 0.06350037  0.19478916  0.00305566 ...  0.06762125  0.11532869
    0.03716978]]

 [[ 0.065717    0.15750356 -0.04845994 ... -0.03104515  0.07409232
    0.08183732]]

 [[ 0.06529244  0.14482117 -0.04807781 ...  0.05233255  0.00911174
    0.02666707]]]
['1721031731_NguyenDongHung' '1721031731_NguyenDongHung'
 '1721031731_NguyenDongHung' ... '151901205_BuiKhanhBinh'
 '151901205_BuiKhanhBinh' '151901205_BuiKhanhBinh']


In [6]:
x.shape

(4700, 1, 128)

In [7]:
y.shape

(4700,)

In [8]:
x = x.reshape(-1, 128)

In [9]:
x.shape

(4700, 128)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [11]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4230, 128), (470, 128), (4230,), (470,))

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [13]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [14]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [15]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.73
Accurency Test = 0.68
F1 Score Train = 0.53
F1 Score Test = 0.51


In [16]:
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [17]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.92
Accurency Test = 0.84
F1 Score Train = 0.84
F1 Score Test = 0.74


In [18]:
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [19]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.79
F1 Score Train = 1.00
F1 Score Test = 0.67


In [20]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [21]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [22]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.83
F1 Score Train = 0.97
F1 Score Test = 0.75


In [23]:
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [24]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   9.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   8.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.1s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [25]:
model_best_estimator = model_grid.best_estimator_

In [26]:
model_grid.best_score_

0.851063829787234

In [27]:
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))